In [1]:
from sklearn.utils.class_weight import compute_sample_weight, compute_class_weight
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import os
import sys
module_path = os.path.abspath(os.path.join('fede'))
if module_path not in sys.path:
    sys.path.append(module_path)
from client import Client
from fedavg import Fedavg
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils import shuffle
from supported_modles import Supported_modles
import random
import utils

In [2]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [3]:
selected_model = Supported_modles.logistic_regression

In [4]:
clients, test_x, test_y = utils.set_data(False,True)
f1_before = []
for client in clients:
    client.init_empty_model(selected_model,0.001)
    client.train_model()
    print(client.y.mean())
    y_hat = client.model.predict(test_x)
    f1_before.append(f1_score(test_y,y_hat, average="binary"))

Creating node1.
Creating node2.
Creating node3.
Creating node4.
Creating node5.
0.55375
0.5617
0.5464
0.548
0.548


In [5]:
f1_before

[0.7672586757821207,
 0.7674626865671642,
 0.7650293666529497,
 0.7674401253450721,
 0.7721495534887167]

In [13]:
selected_model = Supported_modles.SGD_classifier

In [22]:
fedavg = Fedavg("global",0.01)

[Errno 98] Address already in use
Waitiing for a Connection..


In [23]:
fedavg.init_global_model(selected_model, clients[0].model,6)
for client in clients:
    client.init_empty_model(selected_model,0.01)

In [27]:
number_of_rounds = 50
batch_size = 0.2
epochs = 50
prep = StandardScaler() 

for _ in range(number_of_rounds):
    print(_,end=" ")


    applicable_clients = random.sample((clients), random.randint(1, 4))
    applicable_models = []
    applicable_name = []
    round_weights = []
    dataset_size = 0
    

    for client in applicable_clients:
        # print(f'Client name: {client.name}')

        X = client.x
        y = client.y

        X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=batch_size, stratify=y, random_state=random.randint(0,10))
       
        X_train = prep.fit_transform(X_train)

        dataset_size += X_train.shape[0]
        sample_weights = compute_sample_weight('balanced', y=y_train)

        fedavg.load_global_model(client.model, selected_model)
        fedavg.train_local_agent(X_train, y_train, client.model, epochs, sample_weights, selected_model)
        round_weights.append(X_train.shape[0])
        applicable_models.append(client.model)


    round_weights = np.array(round_weights) / dataset_size # calculate weight based on actual dataset size
    # round_weights = weights
    fedavg.update_global_model(applicable_models, round_weights, selected_model)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 

In [28]:
f1_fedavg = []
for client in clients:
    x = prep.transform(test_x)
    y_hat = fedavg.model.predict(x)
    f1_fedavg.append(f1_score(test_y,y_hat))
print(f1_fedavg)

[0.7726140760890388, 0.7726140760890388, 0.7726140760890388, 0.7726140760890388, 0.7726140760890388]


In [29]:
i = 0
for client in clients:
    print(f'Difference {f1_fedavg[i]-f1_before[i]}')
    i += 1

Difference 0.005355400306918057
Difference 0.00515138952187455
Difference 0.007584709436089021
Difference 0.005173950743966649
Difference 0.00046452260032203707
